In [1]:
import pandas as pd
import json
import numpy as np
from dtw import dtw

In [2]:
#algoritmo principale
def DTWDistance(signalA, signalB, channel):
    n,m = len(signalA), len(signalB)
    DTW = np.zeros(shape=(n, m))
    for i in range(0,n):
        DTW[i,0] = np.finfo(dtype=float).max
    for i in range(0,m):
        DTW[0,i] = np.finfo(dtype=float).max
    DTW[0,0] = 0
    
    for i in range(1,n):
        for j in range(1,m):
            const = abs(signalA[i][channel] - signalB[j][channel])
            DTW[i,j] = const + min(DTW[i-1, j], DTW[i, j-1], DTW[i-1, j-1])
    
    return DTW[n - 1, m - 1]

In [3]:
#Algoritmo di scaling
def new_scale_df(data_frame, gyro, accl, pres):
    
    def scale(value, max_, min_):
        return (value - min_) / (max_ - min_)
        
    copy = data_frame.copy() 
    
    for wIndex in range(len(copy)):
        for vIndex in range(len(copy[wIndex])):
            copy[wIndex][vIndex][0] = scale(copy[wIndex][vIndex][0], accl, -accl)
            copy[wIndex][vIndex][1] = scale(copy[wIndex][vIndex][1], accl, -accl)
            copy[wIndex][vIndex][2] = scale(copy[wIndex][vIndex][2], accl, -accl)
            copy[wIndex][vIndex][3] = scale(copy[wIndex][vIndex][3], pres, 0)
            copy[wIndex][vIndex][4] = scale(copy[wIndex][vIndex][4], gyro, -gyro)
            copy[wIndex][vIndex][5] = scale(copy[wIndex][vIndex][5], gyro, -gyro)
            copy[wIndex][vIndex][6] = scale(copy[wIndex][vIndex][6], gyro, -gyro)
    
    return copy

In [4]:
with open('./giovanni_new.json', 'r') as f:
    data_giovanni = json.load(f)
with open('./dario_new.json', 'r') as f:
    data_dario = json.load(f)

In [5]:
scaled_giovanni = new_scale_df(data_giovanni, 250, 2, 1024)
scaled_dario = new_scale_df(data_dario, 250, 2, 1024)

In [6]:
value = [0,0,0,0]
for i in range(7):
    print('indice: ', i)
    l2_norm = lambda x, y: (x[i] - y[i])
    d1, _ , _ , _ = dtw(scaled_giovanni[1], scaled_dario[1], dist=l2_norm)
    print('gio vs dario',d1)
    d2, _ , _ , _ = dtw(scaled_giovanni[2], scaled_dario[2], dist=l2_norm)
    print('gio vs dario',d2)
    d3, _ , _ , _ = dtw(scaled_giovanni[1], scaled_giovanni[2], dist=l2_norm)
    print('gio vs gio',d3)
    d4, _ , _ , _ = dtw(scaled_dario[1], scaled_dario[2], dist=l2_norm)
    print('dario vs dario',d4)
    value[0] += d1
    value[1] += d2
    value[2] += d3
    value[3] += d4
value
    

#l2_norm = lambda x, y: (x[0] - y[0]) ** 2
#d1, cost_matrix, acc_cost_matrix, path = dtw(scaled_giovanni[1], scaled_dario[1], dist=l2_norm)
#d2, _ , _ , _ = dtw(scaled_giovanni[1], scaled_giovanni[2], dist=l2_norm)
#print(d1, d2)

indice:  0
gio vs dario -0.21979188482384798
gio vs dario -0.1808809754495159
gio vs gio -0.20675566169590637
dario vs dario -0.2982841786303629
indice:  1
gio vs dario -0.3121331930894309
gio vs dario -0.23462137621023502
gio vs gio -0.2523709859649125
dario vs dario -0.2548361691419143
indice:  2
gio vs dario -0.46128047018970125
gio vs dario -0.4378430656984788
gio vs gio -0.36800855000000016
dario vs dario -0.43341250000000003
indice:  3
gio vs dario -0.32017488143631434
gio vs dario -0.33553769017980634
gio vs gio -0.2948030884502924
dario vs dario -0.24510429558580857
indice:  4
gio vs dario -0.00013107385907860643
gio vs dario -0.00021353045643154482
gio vs gio -0.0007422532023391677
dario vs dario -0.0007021898415841757
indice:  5
gio vs dario -0.026468730723577236
gio vs dario -0.0504300155546335
gio vs gio -0.038909619920467854
dario vs dario -0.053004485716171695
indice:  6
gio vs dario -0.08101082151761525
gio vs dario -0.08912470064177039
gio vs gio -0.08551405766783635
da

[-1.4209910556395655,
 -1.3286513541908715,
 -1.2471042169017548,
 -1.404840932978548]

In [10]:
matrix = np.zeros(shape=(len(scaled_dario),len(scaled_dario) + len(scaled_giovanni)))

for i in range(len(scaled_dario)):
    for j in range(len(scaled_dario)):
        for s in range(7):
            l2_norm = lambda x, y: abs(x[s] - y[s]) 
            d, _ , _ , _ = dtw(scaled_dario[i], scaled_dario[j], dist=l2_norm)
            matrix[i, j] += d
    for j in range(len(scaled_giovanni)):
        for s in range(7):
            l2_norm = lambda x, y: abs(x[s] - y[s])
            d, _ , _ , _ = dtw(scaled_dario[i], scaled_giovanni[j], dist=l2_norm)
            matrix[i, j + len(scaled_dario)] += d
    print(i)
            
matrix
    

0
1
2
3
4
5
6
7
8
9


array([[0.        , 0.10867773, 0.09708277, 0.12313468, 0.12814563,
        0.13034746, 0.12337908, 0.1156891 , 0.12119785, 0.1554327 ,
        0.11552723, 0.15528203, 0.14515211, 0.13955939, 0.17848311,
        0.16266167, 0.13329751, 0.13122668],
       [0.10867773, 0.        , 0.08974159, 0.12239352, 0.1226707 ,
        0.12626777, 0.13218478, 0.10934641, 0.1229695 , 0.16257821,
        0.10144742, 0.11474823, 0.11145702, 0.13715662, 0.14042746,
        0.15401741, 0.11736181, 0.12466017],
       [0.09708277, 0.08974159, 0.        , 0.12255242, 0.12521944,
        0.13155389, 0.12919551, 0.11333758, 0.12530808, 0.13675898,
        0.08502392, 0.11346048, 0.11141996, 0.11576143, 0.14938799,
        0.15064323, 0.10252908, 0.11526057],
       [0.12313468, 0.12239352, 0.12255242, 0.        , 0.12337697,
        0.11925764, 0.12085462, 0.12237258, 0.1188649 , 0.19787048,
        0.14605787, 0.16054576, 0.13341261, 0.15310624, 0.15221994,
        0.16359152, 0.15391895, 0.14613896],
    

In [14]:
d = pd.DataFrame(matrix)
for i in range(len(matrix)):
    print(i,':',(matrix[i]).argmax(), max(matrix[i]))

0 : 14 0.1784831130510638
1 : 9 0.16257821003817727
2 : 15 0.15064323254943504
3 : 9 0.1978704813267543
4 : 9 0.1920632030272512
5 : 9 0.18021850476027398
6 : 15 0.19499665540496458
7 : 9 0.17356726472979217
8 : 11 0.18593144899469494
9 : 14 0.26879626876372537


In [15]:
d

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.000000,0.108678,0.097083,0.123135,0.128146,0.130347,0.123379,0.115689,0.121198,0.155433,0.115527,0.155282,0.145152,0.139559,0.178483,0.162662,0.133298,0.131227
1,0.108678,0.000000,0.089742,0.122394,0.122671,0.126268,0.132185,0.109346,0.122970,0.162578,0.101447,0.114748,0.111457,0.137157,0.140427,0.154017,0.117362,0.124660
2,0.097083,0.089742,0.000000,0.122552,0.125219,0.131554,0.129196,0.113338,0.125308,0.136759,0.085024,0.113460,0.111420,0.115761,0.149388,0.150643,0.102529,0.115261
3,0.123135,0.122394,0.122552,0.000000,0.123377,0.119258,0.120855,0.122373,0.118865,0.197870,0.146058,0.160546,0.133413,0.153106,0.152220,0.163592,0.153919,0.146139
4,0.128146,0.122671,0.125219,0.123377,0.000000,0.123171,0.129962,0.124851,0.122352,0.192063,0.146061,0.167247,0.138832,0.151093,0.150582,0.162071,0.160052,0.147011
5,0.130347,0.126268,0.131554,0.119258,0.123171,0.000000,0.116857,0.120115,0.113227,0.180219,0.140482,0.177757,0.137507,0.146686,0.153276,0.173957,0.153480,0.150412
6,0.123379,0.132185,0.129196,0.120855,0.129962,0.116857,0.000000,0.121191,0.105388,0.157894,0.127656,0.182861,0.127024,0.139840,0.157363,0.194997,0.143432,0.156298
7,0.115689,0.109346,0.113338,0.122373,0.124851,0.120115,0.121191,0.000000,0.114797,0.173567,0.132134,0.146758,0.125597,0.141119,0.146966,0.155145,0.142096,0.136121
8,0.121198,0.122970,0.125308,0.118865,0.122352,0.113227,0.105388,0.114797,0.000000,0.154612,0.127362,0.185931,0.124050,0.129856,0.161601,0.184179,0.137207,0.143356
9,0.155433,0.162578,0.136759,0.197870,0.192063,0.180219,0.157894,0.173567,0.154612,0.000000,0.116531,0.186486,0.193211,0.154350,0.268796,0.219160,0.130395,0.178002
